In [ ]:
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt

In [ ]:
P = np.loadtxt('../P.txt')
b = np.loadtxt('../b.txt')
w = np.loadtxt('../w.txt')
points = np.loadtxt('../points.txt')
r = P.dot(w) - b
np.linalg.norm(r)

In [ ]:
volume = sum(w)
centroid = points.transpose().dot(w) / volume

# Set up linprog constraints
A_eq = P.copy()
b_eq = b.copy()
x0 = w.copy()
c = np.zeros(w.size)#np.ones(w.size)

In [ ]:
# Note: linprog has default non-negative bound x >= 0
result = linprog(c, A_eq = A_eq, b_eq=b_eq, x0=x0, method='simplex')
result

In [ ]:
w_new = result.x.copy()

# Check that w_new also satisfies constraints
r_new = P.dot(w_new) - b
print(np.min(w_new))
np.linalg.norm(r_new)

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(w, color="blue", linewidth=2.5, linestyle="-", label="w")
plt.plot(w_new, color="red",  linewidth=2.5, linestyle="-", label="w_new")
plt.legend(loc='upper left', frameon=False)
plt.show()

In [ ]:
num_nonzero_old = len([w_i for w_i in w if w_i > 1e-14])
num_nonzero_new = len([w_i for w_i in w_new if w_i > 1e-14])
print("Num nonzero old: {}".format(num_nonzero_old))
print("Num nonzero new: {}".format(num_nonzero_new))

In [ ]:
# Try the same thing with Google's GLOP solver
from ortools.linear_solver import pywraplp

In [ ]:
solver = pywraplp.Solver('quadrature_lp', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

w_vars = [ solver.NumVar(0.0, float("inf"), "w_{}".format(i)) for i in range(0, len(w))]

# TODO: Add P constraints
def create_P_row_constraint(P, b, i):
    constraint = solver.Constraint(b[i], b[i], 'P_{}'.format(i))
    for j in range(0, P.shape[1]):
        constraint.SetCoefficient(w_vars[j], P[i, j])
        
P_constraints = [ create_P_row_constraint(P, b, i) for i in range(0, P.shape[0]) ]
objective = solver.Objective()
# for w_i in w_vars:
#     objective.SetCoefficient(w_i, 1)

solver.Solve()
objective.Value()

w_glop = np.array([ w_i.solution_value() for w_i in w_vars ])

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(w, color="blue", linewidth=2.5, linestyle="-", label="w")
plt.plot(w_new, color="red",  linewidth=2.5, linestyle="-", label="w_linprog")
plt.plot(w_new, color="green",  linewidth=2.5, linestyle="-", label="w_glop")
plt.legend(loc='upper left', frameon=False)
plt.show()

In [ ]:
num_nonzero_glop = len([w_i for w_i in w_glop if w_i > 1e-14])
print("Num nonzero glop: {}".format(num_nonzero_glop))

In [ ]:
from bokeh.plotting import figure, show, output_file, output_notebook
from math import sqrt, pi

output_notebook()

def plot_points(points, weights):
    x = points[:, 0]
    y = points[:, 1]
    print(sum(weights))
    radii = [ sqrt(w_i / pi) for w_i in weights ]
    colors = [
        "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
    ]

    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    p = figure(tools=TOOLS)

    p.scatter(x, y, radius=radii,
              fill_color=colors, fill_alpha=0.6,
              line_color=None)

    show(p)

In [ ]:
plot_points(points, w_new)

In [ ]:
plot_points(points, w_glop)

In [ ]:
plot_points(points, w)